In [ ]:
import requests
import pandas as pd
import pandas_ta as ta
import seaborn as sns
import matplotlib.pyplot as plt
import json
import time
import os


plt.style.use('seaborn-darkgrid')
plt.style.use('seaborn-notebook')
plt.style.use('dark_background')


class portfolio_insider():
    def __init__(self):
        with open('portfolio_insider_configs.json','r') as file:
            self.config_data = json.load(file)
            self.api_key = self.config_data['API-KEY'].strip()
            
        try:
            self.filename = input('Stock-List Name? ').strip()
        except FileNotFoundError:
            return
        
        self.returns_df = pd.DataFrame()
        self.atr_sma_df = pd.DataFrame()

        try:
            with open(self.filename, 'r') as file:
                self.stocks = [line.strip() for line in file.readlines()]
        except FileNotFoundError:
            print(f'No such file as {self.filename}')
            print('Please create your portfolio with portfolio_manager.ipynb first.')
            return
        
        self.atr_length = int(input('ATR Lenght? '))
        
        if self.atr_length > len(self.stocks):
            self.atr_length = len(self.stocks)
            
        self.save = bool(int(input('Save Plots? (0/1) ')))
        self.startdate = input('Startdate? (YYYY-MM-DD) ')
        self.performance_df = pd.DataFrame(columns=[symbol for symbol in self.stocks])
                                           
        print(f'This may take up to {len(self.stocks) * 12 + 3}s.\n')

        for symbol in self.stocks:
            print(f"Requesting data for {symbol}...")
            url = (
                f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={self.api_key}')

            response = requests.get(url)
            data = response.json()
            
            if response.status_code != 200 or "Error Message" in data:
                print(f"Failed to retrieve data for {symbol}. Skipping...")
                continue

            if "Note" in data:
                print(f"API limit reached: {data['Note']}")
                return

            if "Time Series (Daily)" not in data:
                print(f"Error at {symbol}: {data}")
                continue
            
            ts_data = data["Time Series (Daily)"]
            df = pd.DataFrame.from_dict(ts_data, orient='index')
            df.index = pd.to_datetime(df.index)
            df = df.rename(columns={'1. open': 'Open','2. high': 'High','3. low': 'Low','4. close': 'Close'})

            try:
                df = df[df.index > self.startdate][['Open', 'High', 'Low', 'Close']].copy()
                df[['Open', 'High', 'Low', 'Close']] = df[['Open', 'High', 'Low', 'Close']].astype(float)
            except Exception as e:
                print(f"Error converting {symbol} data: {e}")
                continue
            
            returns = df['Close'].pct_change().dropna()
            cumulative = (1 + returns).cumprod() - 1
            scaled = cumulative - cumulative.iloc[0]
            self.performance_df[symbol] = scaled
            
            if self.returns_df.empty:
                self.returns_df = returns.to_frame(name=symbol)
            else:
                self.returns_df = self.returns_df.join(returns.to_frame(name=symbol), how='inner')

            if len(df) >= atr_length:
                atr = ta.atr(high=df['High'], low=df['Low'], close=df['Close'], length=atr_length)
                atr_sma = atr.rolling(window=atr_length).mean()
                self.atr_sma_df[symbol] = atr_sma
        
            time.sleep(12)

    def get_correlations(self):
        if self.returns_df.empty:
            print("No return data available.")
            return

        corr = self.returns_df.corr()
        
        sns.heatmap(corr, cbar=False, cmap='coolwarm', annot=True, annot_kws={'size':6}, linewidths=0, square=True)
        plt.title("Correlation Matrix",size=14,fontweight='bold')
        plt.xticks(fontsize=6) 
        plt.yticks(fontsize=6)
        
        if self.save == True:
            plt.savefig(f'{self.filename}_correlation.png',dpi=300)
            
        plt.show()

    def get_atr(self):
        if self.atr_sma_df.empty:
            print("No ATR data available.")
            return
            
        for symbol in self.atr_sma_df.columns:
            plt.plot(self.atr_sma_df[symbol], label=symbol, linewidth=0.33, alpha=0.5)

        mean_atr = self.atr_sma_df.mean(axis=1)
        
        plt.plot(mean_atr, label='Mean ATR', linewidth=3, color='red')
        plt.title(f'Smoothed ATR ({self.atr_length}d)',size=14,fontweight='bold')
        plt.xlabel('Date',fontsize=6)
        plt.ylabel('ATR',fontsize=6)
        plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))
        plt.grid(True, alpha=0.1)
        
        if self.save == True:
            plt.savefig(f'{self.filename}_atr.png',dpi=300)
            
        plt.show()
        
    def get_performance(self):
        if self.returns_df.empty:
            print("No return data available.")
            return
        
        if self.performance_df.empty:
            print("No return data available.")
            return
        
        plt.plot(self.performance_df.sum(axis=1)*100/len(self.stocks),linewidth=3,color='r',label=f'Performance: {(self.performance_df.sum(axis=1)[len(self.performance_df)]/len(self.stocks)*100):.2f}%')
        plt.axhline(self.performance_df.sum(axis=1)[len(self.performance_df)]/len(self.stocks)*100,linewidth=3, linestyle='dotted',color='r')
         
        for symbol in self.stocks:
            plt.plot(self.performance_df[symbol]*100,label=symbol,linewidth=0.33,alpha=0.5)
            
        plt.title('Performance',size=14,fontweight='bold')
        plt.xlabel('Date',fontsize=6)
        plt.ylabel('Cumulative Return (%)',fontsize=6)
        plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))
        plt.grid(True, alpha=0.1)
        
        if self.save == True:
            plt.savefig(f'{self.filename}_performance.png',dpi=300)
            
        plt.show()
        
        
if __name__ == "__main__":
    portfolio = portfolio_insider()
    portfolio.get_correlations()
    portfolio.get_atr()
    portfolio.get_performance()